Mounted Google Drive



Loaded the dataset into a Pandas DataFrame.




In [78]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Reads the CSV file into df

In [79]:
import pandas as pd
file_path = "/content/drive/MyDrive/TRADES_CopyTr_90D_ROI.csv"
df = pd.read_csv(file_path)
print(df.info())
print(df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Port_IDs       150 non-null    int64 
 1   Trade_History  149 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.5+ KB
None
              Port_IDs                                      Trade_History
0  3925368433214965504  [{'time': 1718899656000, 'symbol': 'SOLUSDT', ...
1  4002413037164645377  [{'time': 1718980078000, 'symbol': 'NEARUSDT',...
2  3923766029921022977  [{'time': 1718677164000, 'symbol': 'ETHUSDT', ...
3  3994879592543698688  [{'time': 1718678214000, 'symbol': 'ETHUSDT', ...
4  3926423286576838657  [{'time': 1718979615000, 'symbol': 'ETHUSDT', ...


In [80]:
print(df.isnull().sum())

Port_IDs         0
Trade_History    1
dtype: int64


1. Replace Missing Values (NaN → "[]")
2. Convert String to List of Dictionaries
3. Explode Trade Lists into Individual Rows
4. Remove Empty Trades
5. Extract Dictionary Data into Columns
6. Drop the Original Trade_History Column
7. Print the Processed Data
  







  



In [81]:
import pandas as pd
import ast
df['Trade_History'] = df['Trade_History'].fillna("[]")
df['Trade_History'] = df['Trade_History'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
trades_df = df.explode('Trade_History')
trades_df = trades_df[trades_df['Trade_History'].notna()]
trades_df = pd.concat([trades_df.drop(['Trade_History'], axis=1), trades_df['Trade_History'].apply(pd.Series)], axis=1)

print(trades_df.head())


              Port_IDs           time    symbol side      price       fee  \
0  3925368433214965504  1718899656000   SOLUSDT  BUY  132.53700 -0.994027   
0  3925368433214965504  1718899618000  DOGEUSDT  BUY    0.12182 -0.279796   
0  3925368433214965504  1718899618000  DOGEUSDT  BUY    0.12182 -0.039494   
0  3925368433214965504  1718899616000  DOGEUSDT  BUY    0.12182 -0.008284   
0  3925368433214965504  1718899616000  DOGEUSDT  BUY    0.12182 -0.046109   

  feeAsset    quantity quantityAsset  realizedProfit realizedProfitAsset  \
0     USDT  1988.05500          USDT             0.0                USDT   
0     USDT  1398.98088          USDT             0.0                USDT   
0     USDT   197.47022          USDT             0.0                USDT   
0     USDT    16.56752          USDT             0.0                USDT   
0     USDT    92.21774          USDT             0.0                USDT   

  baseAsset      qty positionSide  activeBuy  
0       SOL     15.0         LONG



1. Grouping Trades by Port_IDs and Summing realizedProfit
2. Renaming realizedProfit Column to Total_PnL



In [82]:
pnl_df = trades_df.groupby("Port_IDs")["realizedProfit"].sum().reset_index()
pnl_df.rename(columns={"realizedProfit": "Total_PnL"}, inplace=True)


In [83]:
pnl_df

,Port_IDs,Total_PnL
0,3672754654734989568,566.597660
1,3733192481840423936,2923.977200
2,3768170840939476993,243.668899
3,3784403294629753856,2521.814305
4,3786761687746711808,205.021400
...,...,...
144,4039279455324236544,1038.807419
145,4040382575336130560,0.000000
146,4040843843196854529,2151.704060
147,4041804592937345281,-776.343000


In [84]:
port_id = 3672754654734989568
filtered_trades = trades_df[trades_df["Port_IDs"] == port_id]

filtered_trades[["Port_IDs", "realizedProfit"]]


,Port_IDs,realizedProfit
86,3672754654734989568,9.462000
86,3672754654734989568,1.743700
86,3672754654734989568,0.191127
86,3672754654734989568,0.047982
86,3672754654734989568,1.835691
...,...,...
86,3672754654734989568,0.000000
86,3672754654734989568,0.000000
86,3672754654734989568,0.000000
86,3672754654734989568,0.419900




1. Calculate Total_Investment per Account
2. Renamed Column to Total_Investment
3. Merge Investment Data with PnL Data
4. Calculate ROI (Return on Investment) for Each Account
   Formula:
          ROI=(TOTAL PNL/TOTAL INVESTMENT) x 100





In [85]:
investment_df = trades_df[trades_df["side"] == "BUY"].groupby("Port_IDs")["quantity"].sum().reset_index()
investment_df.rename(columns={"quantity": "Total_Investment"}, inplace=True)
roi_df = pnl_df.merge(investment_df, on="Port_IDs", how="left")
roi_df["ROI"] = (roi_df["Total_PnL"] / roi_df["Total_Investment"]) * 100


In [86]:
investment_df.head()

,Port_IDs,Total_Investment
0,3672754654734989568,61735.947452
1,3733192481840423936,580774.260700
2,3768170840939476993,766.603827
3,3784403294629753856,377877.824072
4,3786761687746711808,32724.336700


In [87]:
roi_df.head()

,Port_IDs,Total_PnL,Total_Investment,ROI
0,3672754654734989568,566.597660,61735.947452,0.917776
1,3733192481840423936,2923.977200,580774.260700,0.503462
2,3768170840939476993,243.668899,766.603827,31.785505
3,3784403294629753856,2521.814305,377877.824072,0.667362
4,3786761687746711808,205.021400,32724.336700,0.626510


The Sharpe Ratio is a measure of risk-adjusted returns, which helps evaluate how profitable an account is relative to its risk (volatility).


1. Calculated Mean & Standard Deviation of PnL per Account
2.  Renamed Columns for Clarity like "mean" → "Mean_PnL"
"std" → "Std_PnL"
3. Dividing by 0 would cause an error, so we replace 0 with a very small number (1e-9).
4. Calculated Sharpe Ratio
        FORMULA:
               Mean PnL/Std PnL



In [88]:
sharpe_df = trades_df.groupby("Port_IDs")["realizedProfit"].agg(["mean", "std"]).reset_index()
sharpe_df.rename(columns={"mean": "Mean_PnL", "std": "Std_PnL"}, inplace=True)

sharpe_df["Std_PnL"].replace(0, 1e-9, inplace=True)

sharpe_df["Sharpe_Ratio"] = sharpe_df["Mean_PnL"] / sharpe_df["Std_PnL"]


<ipython-input-88-4e4c55f18a4b>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sharpe_df["Std_PnL"].replace(0, 1e-9, inplace=True)


In [89]:
sharpe_df.head()

,Port_IDs,Mean_PnL,Std_PnL,Sharpe_Ratio
0,3672754654734989568,1.195354,6.451799,0.185274
1,3733192481840423936,4.243799,70.418447,0.060265
2,3768170840939476993,17.404921,41.022499,0.424277
3,3784403294629753856,0.416829,3.910762,0.106585
4,3786761687746711808,2.500261,11.620986,0.215150


Maximum Drawdown (MDD) is a measure of downside risk that shows the worst loss from a peak before a recovery. It helps identify how much an account has dropped from its highest value before rebounding.


1. Apply max_drawdown to Each Account
2. Renamed realizedProfit to Max_Drawdown



In [90]:
def max_drawdown(profit_series):
    cumulative_returns = profit_series.cumsum()
    peak = cumulative_returns.cummax()
    drawdown = (cumulative_returns - peak) / peak
    return drawdown.min()

mdd_df = trades_df.groupby("Port_IDs")["realizedProfit"].apply(max_drawdown).reset_index()
mdd_df.rename(columns={"realizedProfit": "Max_Drawdown"}, inplace=True)


In [91]:
mdd_df.head()

,Port_IDs,Max_Drawdown
0,3672754654734989568,-0.460781
1,3733192481840423936,-0.760355
2,3768170840939476993,0.000000
3,3784403294629753856,-0.177261
4,3786761687746711808,-2.318212




1. Count Total Trades per Account
2. Count Winning Trades per Account
3. Merge Win Trades with Total Trades
4. Calculate Win Rate (%)
       
       FORMULA: (WIN POSITION/TOTAL POSITION) X 100




In [92]:
total_positions_df = trades_df.groupby("Port_IDs")["realizedProfit"].count().reset_index()
total_positions_df.rename(columns={"realizedProfit": "Total_Positions"}, inplace=True)

win_positions_df = trades_df[trades_df["realizedProfit"] > 0].groupby("Port_IDs")["realizedProfit"].count().reset_index()
win_positions_df.rename(columns={"realizedProfit": "Win_Positions"}, inplace=True)

win_rate_df = total_positions_df.merge(win_positions_df, on="Port_IDs", how="left").fillna(0)

win_rate_df["Win_Rate"] = (win_rate_df["Win_Positions"] / win_rate_df["Total_Positions"]) * 100


In [93]:
win_rate_df.head()

,Port_IDs,Total_Positions,Win_Positions,Win_Rate
0,3672754654734989568,474,210.0,44.303797
1,3733192481840423936,689,553.0,80.261248
2,3768170840939476993,14,6.0,42.857143
3,3784403294629753856,6050,1829.0,30.231405
4,3786761687746711808,82,37.0,45.121951




1. Merge All DataFrames
2. Define Weightages for Each Metric
3. Calculate Weighted Score for Each Account
4. Rank Accounts Based on Score



In [94]:
final_df = roi_df.merge(sharpe_df, on="Port_IDs", how="left")
final_df = final_df.merge(mdd_df, on="Port_IDs", how="left")
final_df = final_df.merge(win_rate_df, on="Port_IDs", how="left")


weights = {
    "ROI": 0.4,
    "Sharpe_Ratio": 0.3,
    "Max_Drawdown": -0.2,
    "Win_Rate": 0.1
}


final_df["Score"] = (
    final_df["ROI"] * weights["ROI"] +
    final_df["Sharpe_Ratio"] * weights["Sharpe_Ratio"] +
    final_df["Max_Drawdown"] * weights["Max_Drawdown"] +
    final_df["Win_Rate"] * weights["Win_Rate"]
)

final_df.head()



,Port_IDs,Total_PnL,Total_Investment,ROI,Mean_PnL,Std_PnL,Sharpe_Ratio,Max_Drawdown,Total_Positions,Win_Positions,Win_Rate,Score
0,3672754654734989568,566.597660,61735.947452,0.917776,1.195354,6.451799,0.185274,-0.460781,474,210.0,44.303797,4.945229
1,3733192481840423936,2923.977200,580774.260700,0.503462,4.243799,70.418447,0.060265,-0.760355,689,553.0,80.261248,8.397660
2,3768170840939476993,243.668899,766.603827,31.785505,17.404921,41.022499,0.424277,0.000000,14,6.0,42.857143,17.127200
3,3784403294629753856,2521.814305,377877.824072,0.667362,0.416829,3.910762,0.106585,-0.177261,6050,1829.0,30.231405,3.357513
4,3786761687746711808,205.021400,32724.336700,0.626510,2.500261,11.620986,0.215150,-2.318212,82,37.0,45.121951,5.290987


In [95]:
top_2_accounts = final_df.head(2)
top_2_accounts

,Port_IDs,Total_PnL,Total_Investment,ROI,Mean_PnL,Std_PnL,Sharpe_Ratio,Max_Drawdown,Total_Positions,Win_Positions,Win_Rate,Score
0,3672754654734989568,566.59766,61735.947452,0.917776,1.195354,6.451799,0.185274,-0.460781,474,210.0,44.303797,4.945229
1,3733192481840423936,2923.97720,580774.260700,0.503462,4.243799,70.418447,0.060265,-0.760355,689,553.0,80.261248,8.397660




1. Replace inf and -inf with NaN
2. Replace NaN with 0



In [96]:
import numpy as np
final_df.replace([np.inf, -np.inf], np.nan, inplace=True)
final_df.fillna(0, inplace=True)


In [97]:
final_df

,Port_IDs,Total_PnL,Total_Investment,ROI,Mean_PnL,Std_PnL,Sharpe_Ratio,Max_Drawdown,Total_Positions,Win_Positions,Win_Rate,Score
0,3672754654734989568,566.597660,61735.947452,0.917776,1.195354,6.451799e+00,0.185274,-0.460781,474,210.0,44.303797,4.945229
1,3733192481840423936,2923.977200,580774.260700,0.503462,4.243799,7.041845e+01,0.060265,-0.760355,689,553.0,80.261248,8.397660
2,3768170840939476993,243.668899,766.603827,31.785505,17.404921,4.102250e+01,0.424277,0.000000,14,6.0,42.857143,17.127200
3,3784403294629753856,2521.814305,377877.824072,0.667362,0.416829,3.910762e+00,0.106585,-0.177261,6050,1829.0,30.231405,3.357513
4,3786761687746711808,205.021400,32724.336700,0.626510,2.500261,1.162099e+01,0.215150,-2.318212,82,37.0,45.121951,5.290987
...,...,...,...,...,...,...,...,...,...,...,...,...
144,4039279455324236544,1038.807419,65391.882080,1.588588,3.176781,6.734983e+00,0.471684,0.000000,327,181.0,55.351682,6.312108
145,4040382575336130560,0.000000,19559.429400,0.000000,0.000000,1.000000e-09,0.000000,0.000000,76,0.0,0.000000,0.000000
146,4040843843196854529,2151.704060,103854.024000,2.071854,36.469560,1.067399e+02,0.341668,0.000000,59,19.0,32.203390,4.151581
147,4041804592937345281,-776.343000,290060.374240,-0.267649,-2.109628,2.707110e+01,-0.077929,-46.067384,368,85.0,23.097826,11.392821


In [98]:
final_df["ROI"] = final_df.apply(
    lambda row: row["Total_PnL"] / row["Total_Investment"] if row["Total_Investment"] != 0 else 0, axis=1
)


In [99]:
final_df["Sharpe_Ratio"] = final_df.apply(
    lambda row: row["Mean_PnL"] / row["Std_PnL"] if row["Std_PnL"] != 0 else 0, axis=1
)


In [100]:
final_df["Win_Rate"] = final_df.apply(
    lambda row: (row["Win_Positions"] / row["Total_Positions"]) * 100 if row["Total_Positions"] != 0 else 0, axis=1
)


In [101]:
final_df = final_df.sort_values(by="Score", ascending=False)

In [102]:
final_df.head(2)

,Port_IDs,Total_PnL,Total_Investment,ROI,Mean_PnL,Std_PnL,Sharpe_Ratio,Max_Drawdown,Total_Positions,Win_Positions,Win_Rate,Score
52,3966142151544441601,4883.800776,801217.862632,0.006095,2.527847,48.159777,0.052489,-181.38151,1932,1028.0,53.209110,41.856778
2,3768170840939476993,243.668899,766.603827,0.317855,17.404921,41.022499,0.424277,0.00000,14,6.0,42.857143,17.127200


Selected Top 20 accounts and exported to Top_20_Accounts.csv.

In [103]:
top_20_accounts_final = final_df.head(20)
top_20_accounts_final

,Port_IDs,Total_PnL,Total_Investment,ROI,Mean_PnL,Std_PnL,Sharpe_Ratio,Max_Drawdown,Total_Positions,Win_Positions,Win_Rate,Score
52,3966142151544441601,4883.800776,8.012179e+05,0.006095,2.527847,48.159777,0.052489,-181.381510,1932,1028.0,53.209110,41.856778
2,3768170840939476993,243.668899,7.666038e+02,0.317855,17.404921,41.022499,0.424277,0.000000,14,6.0,42.857143,17.127200
12,3879821005658659073,4268.006561,9.631928e+04,0.044311,4.894503,37.379702,0.130940,-42.586646,872,289.0,33.142202,13.643273
8,3826087012661391104,532.656974,3.032332e+03,0.175659,4.932009,7.305582,0.675101,-0.050348,108,63.0,58.333333,13.072301
80,4004410127575640832,2905.763867,2.382449e+05,0.012197,5.731290,30.311737,0.189078,-44.061727,507,175.0,34.516765,12.808607
50,3960874214179953664,19567.471286,4.261385e+06,0.004592,8.183802,50.223455,0.162948,-34.536210,2391,1233.0,51.568381,12.296637
147,4041804592937345281,-776.343000,2.900604e+05,-0.002676,-2.109628,27.071103,-0.077929,-46.067384,368,85.0,23.097826,11.392821
62,3986814617275053313,16337.461881,9.690091e+05,0.016860,4.596922,13.939083,0.329787,-0.000149,3554,2780.0,78.221722,8.595537
1,3733192481840423936,2923.977200,5.807743e+05,0.005035,4.243799,70.418447,0.060265,-0.760355,689,553.0,80.261248,8.397660
47,3956048468100538880,1373.564890,5.103102e+04,0.026916,49.055889,93.464378,0.524862,0.000000,28,20.0,71.428571,8.376967


safely exported top_20_accounts_final to CSV

In [104]:
top_20_accounts_final.to_csv("Top_20_Accounts.csv", index=False)
